In [ ]:
import json
import os
import random
from torch.utils.data import Dataset
from pycocotools.coco import COCO
from pycocotools import mask as maskUtils
from lvis import LVIS
from tqdm import tqdm
from lvis import LVISEval, LVISResults
import numpy as np

In [ ]:
print("Begin Evaluation!")
lvis_ann = LVIS('Visual-RFT/data/lvis_v1_val.json')

In [ ]:
with open('prediction_results.json', 'r', encoding='utf-8') as f:
    lvis_results =  json.load(f)

In [ ]:
lvis_results = LVISResults(lvis_ann, lvis_results, max_dets=300)
lvis_eval = LVISEval(lvis_ann, lvis_results, "bbox")
lvis_eval.run()
lvis_eval.print_results()

In [ ]:
selected_cate = ['horse_buggy', 'die', 'kitchen_table', 'omelet', 'papaya', 'stepladder']

In [ ]:
selected_cate = ['casserole', 'die', 'egg_roll', 'futon', 'garbage', 'handsaw', 'hippopotamus', 'kitchen_table', 'mallet', 'omelet', 'shot_glass', 'stepladder', 'sugar_bowl']

In [ ]:
# mAP and AP for selected_cate
precisions = lvis_eval.eval['precision']
cats = lvis_ann.load_cats(lvis_ann.get_cat_ids())
cats_names = [item['name'] for item in cats]
per_class_maps = []
per_class_results = []
for idx, name in enumerate(cats_names):
    precision = precisions[:, :, idx, :]
    precision = precision[precision > -1]
    ap = np.mean(precision) if precision.size else float("nan")
    per_class_maps.append(float(ap * 100))
    per_class_results.append({name:float(ap * 100)})

ap_count = 0
ap_sum = 0
for item in per_class_results:
    for k,v in item.items():
        if k in selected_cate:
            print(item)
            ap_sum += v
            ap_count += 1
print({'mAP':ap_sum/ap_count})